In [20]:
import polars as pl;
import csv;

Carga de datos desde Drive

In [21]:

from google.colab import drive, files

drive.mount('/content/drive')
log_file_path = '/content/drive/My Drive/Colab_Folder/access.log'
csv_file_path = '/content/drive/My Drive/Colab_Folder/archivo.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Carga de datos desde Kaggle

In [22]:
from google.colab import userdata
import os
!pip install -U -q kaggle

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USER"] = userdata.get('KAGGLE_USER')

!mkdir -p ~/.kaggle
!kaggle datasets download -d sofiamartinez222324/infraestructura-de-datos-espaciales-uy
!unzip filebeat-geoportal-access.csv.zip

403 - Forbidden - Permission 'datasets.get' was denied
unzip:  cannot find or open filebeat-geoportal-access.csv.zip, filebeat-geoportal-access.csv.zip.zip or filebeat-geoportal-access.csv.zip.ZIP.


Convertir archivos .logs a .csv para poder trabajar con Polars

In [23]:
# Abro el archivo original .log
with open(log_file_path, 'r') as file:
    log_data = file.readlines()

# Los encabezados para el archivo .csv
headers = ["ip", "timestamp", "request_method", "url", "http_version", "status_code", "size", "user_agent"]

# Lo leo linea por linea y voy guardando la info en el archivo.csv
with open(csv_file_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(headers)

    for line in log_data:
        parts = line.split(' ')

        try:
            ip = parts[0]
            timestamp = ' '.join(parts[3:5]).strip('[]')
            request_method = parts[5].strip('"') if parts[5] != '"-"' else None
            url = parts[6] if len(parts) > 6 else None
            http_version = parts[7].strip('"') if len(parts) > 7 else None
            status_code = int(parts[8]) if len(parts) > 8 and parts[8].isdigit() else None
            size = int(parts[9]) if len(parts) > 9 and parts[9].isdigit() else None
            user_agent = ' '.join(parts[11:]).strip('"') if len(parts) > 11 else None

            csvwriter.writerow([ip, timestamp, request_method, url, http_version, status_code, size, user_agent])

        except (IndexError, ValueError) as e:
            print(f"Error procesando la línea: {line.strip()} - Error: {e}")
            csvwriter.writerow([ip, timestamp, None, None, None, None, None, None])

## Funciones Auxiliares

#### **filtrar_jcemediabox:**
Estas URLs están asociadas con JCEMediaBox, un plugin de Joomla que facilita la incorporación de lightboxes para mostrar imágenes, vídeos y otros tipos de medios de una manera atractiva y funcional. Los archivos .js contienen el código JavaScript necesario para hacer funcionar el lightbox, mientras que los archivos .css contienen los estilos que determinan la apariencia del lightbox en la página web.

---

#### **filtrar_css y filtrar_js:**
El objetivo de nuestro análisis es entender el tráfico relacionado con la funcionalidad del sitio y excluir contenido estático como .css y .js. Por lo que decidimos filtrar estos archivos. Si quisiéramos hacer un análisis más completo del tráfico, deberíamos mantener estos archivos.

Pero los archivos .css y .js son muy frecuentes en los logs y filtrarlos puede reducir el "ruido" en los datos, permitiéndonos enfocarnos en las solicitudes más relevantes.


In [24]:
##### Funciones auxiliares - Limpieza de datos
def obtener_total_registros():
    return df.height

def contar_registros_url(df, regex):
    return df.filter(pl.col("url").str.contains(regex)).height

def contar_registros_user_agent(df, regex):
    return df.filter(pl.col("user_agent").str.contains(regex)).height


# Robos y Crawlers
def contar_filtros_robots(df):
    total_registros = obtener_total_registros()

    registros_googlebot = contar_registros_user_agent(df, 'Googlebot')
    registros_baiduspider = contar_registros_user_agent(df, 'Baiduspider')
    registros_agesic_crawler = contar_registros_user_agent(df, 'agesic-crawler')

    porcentaje_googlebot = (registros_googlebot / total_registros) * 100
    porcentaje_baiduspider = (registros_baiduspider / total_registros) * 100
    porcentaje_agesic_crawler = (registros_agesic_crawler / total_registros) * 100

    print(f"Registros 'Googlebot': {registros_googlebot} ({porcentaje_googlebot:.2f}%)")
    print(f"Registros 'Baiduspider': {registros_baiduspider} ({porcentaje_baiduspider:.2f}%)")
    print(f"Registros 'agesic-crawler': {registros_agesic_crawler} ({porcentaje_agesic_crawler:.2f}%)")

def filtrar_googlebot(df):
    return df.filter(~pl.col("user_agent").str.contains('Googlebot'))

def filtrar_baiduspider(df):
    return df.filter(~pl.col("user_agent").str.contains('Baiduspider'))

def filtrar_agesic_crawler(df):
    return df.filter(~pl.col("user_agent").str.contains('agesic-crawler'))

def filtrar_robots_y_crawlers(df):
    contar_filtros_robots(df)
    df = filtrar_googlebot(df)
    df = filtrar_baiduspider(df)
    df = filtrar_agesic_crawler(df)
    return df


# Archivos estáticos
def contar_filtros_archivos_estaticos(df):
    total_registros = obtener_total_registros()

    registros_jcemediabox = contar_registros_url(df, '/plugins/system/jcemediabox/')
    registros_css = contar_registros_url(df, r'\.css$')
    registros_js = contar_registros_url(df, r'\.js$')
    registros_png = contar_registros_url(df, r'\.png$')
    registros_jpg = contar_registros_url(df, r'\.jpg$')
    registros_gif = contar_registros_url(df, r'\.gif$')
    registros_favicon = contar_registros_url(df, r'favicon\.ico$')

    porcentaje_jcemediabox = (registros_jcemediabox / total_registros) * 100
    porcentaje_css = (registros_css / total_registros) * 100
    porcentaje_js = (registros_js / total_registros) * 100
    porcentaje_png = (registros_png / total_registros) * 100
    porcentaje_jpg = (registros_jpg / total_registros) * 100
    porcentaje_gif = (registros_gif / total_registros) * 100
    porcentaje_favicon = (registros_favicon / total_registros) * 100

    print(f"Registros 'jcemediabox': {registros_jcemediabox} ({porcentaje_jcemediabox:.2f}%)")
    print(f"Registros '.css': {registros_css} ({porcentaje_css:.2f}%)")
    print(f"Registros '.js': {registros_js} ({porcentaje_js:.2f}%)")
    print(f"Registros '.png': {registros_png} ({porcentaje_png:.2f}%)")
    print(f"Registros '.jpg': {registros_jpg} ({porcentaje_jpg:.2f}%)")
    print(f"Registros '.gif': {registros_gif} ({porcentaje_gif:.2f}%)")
    print(f"Registros 'favicon.ico': {registros_favicon} ({porcentaje_favicon:.2f}%)")

def filtar_jcemediabox(df):
    return df.filter(~pl.col("url").str.contains('/plugins/system/jcemediabox/'))

def filtrar_css(df):
    regex = r'\.css$'
    return df.filter(~pl.col("url").str.contains(regex))

def filtrar_js(df):
    regex = r'\.js$'
    return df.filter(~pl.col("url").str.contains(regex))

def filtrar_png(df):
    regex = r'\.png$'
    return df.filter(~pl.col("url").str.contains(regex))

def filtrar_jpg(df):
    regex = r'\.jpg$'
    return df.filter(~pl.col("url").str.contains(regex))

def filtrar_gif(df):
    regex = r'\.gif$'
    return df.filter(~pl.col("url").str.contains(r'\.gif$'))

def filtrar_favicon(df):
    regex = r'favicon\.ico$'
    return df.filter(~pl.col("url").str.contains(r'favicon\.ico$'))

def filtrar_datos(df):
    contar_filtros_archivos_estaticos(df)
    df = filtar_jcemediabox(df)
    df = filtrar_css(df)
    df = filtrar_js(df)
    df = filtrar_png(df)
    df = filtrar_jpg(df)
    df = filtrar_gif(df)
    df = filtrar_favicon(df)
    return df

def formatear_fecha(df):
    return df.with_columns([pl.col("timestamp").str.strptime(pl.Datetime, "%d/%b/%Y:%H:%M:%S %z", strict=False).dt.strftime("%d-%m-%Y %H:%M:%S").alias("timestamp")])

## Main

In [25]:
df = pl.read_csv(csv_file_path)

#### Reconocimiento del data frame
Veamos cómo está compuesto el dataframe

In [26]:
df.schema

Schema([('ip', String),
        ('timestamp', String),
        ('request_method', String),
        ('url', String),
        ('http_version', String),
        ('status_code', Int64),
        ('size', Int64),
        ('user_agent', String)])

Mostramos las primeras 5 filas del dataframe

In [27]:
df.head(5)

ip,timestamp,request_method,url,http_version,status_code,size,user_agent
str,str,str,str,str,i64,i64,str
"""127.0.0.1""","""02/Jun/2013:06:48:17 -0300""","""OPTIONS""","""*""","""HTTP/1.0""",200,167,"""Apache/2.2.14 (Ubuntu) (intern…"
"""127.0.0.1""","""02/Jun/2013:06:48:17 -0300""","""OPTIONS""","""*""","""HTTP/1.0""",200,167,"""Apache/2.2.14 (Ubuntu) (intern…"
"""127.0.0.1""","""02/Jun/2013:06:48:17 -0300""","""OPTIONS""","""*""","""HTTP/1.0""",200,167,"""Apache/2.2.14 (Ubuntu) (intern…"
"""180.76.5.150""","""02/Jun/2013:06:49:39 -0300""","""GET""","""/index.php/component/jevents/s…","""HTTP/1.1""",200,8842,"""Mozilla/5.0 (compatible; Baidu…"
"""190.64.2.162""","""02/Jun/2013:06:51:40 -0300""","""GET""","""/robots.txt""","""HTTP/1.0""",200,718,"""agesic-crawler (Enterprise; T3…"


Veremos la cantidad de registros con los que cuenta el dataset

In [28]:
cantidad_registros = df.select(pl.all().count())
cantidad_registros

ip,timestamp,request_method,url,http_version,status_code,size,user_agent
u32,u32,u32,u32,u32,u32,u32,u32
207232,207232,206769,207232,207232,206768,206768,206768


Podemos notar que para las columnas de request_method, status_code, size y user_agent se muestra una cantidad de registros menor comparado con las columnas de ip, timestamp, url y http_version.

Veremos con que cantidad de valores vacíos nos encontramos en el df, para verificar si esta es la razón de que nos aparezcan registros faltantes.

In [29]:
cantidad_nulos = df.null_count()
cantidad_nulos

ip,timestamp,request_method,url,http_version,status_code,size,user_agent
u32,u32,u32,u32,u32,u32,u32,u32
0,0,463,0,0,464,464,464


In [30]:
total_registros = cantidad_registros + cantidad_nulos
total_registros

ip,timestamp,request_method,url,http_version,status_code,size,user_agent
u32,u32,u32,u32,u32,u32,u32,u32
207232,207232,207232,207232,207232,207232,207232,207232


Como sospechábamos, la diferencia entre las cantidades de registros se debe a los valores en nulo.

El porcentaje de valores nulos por columna es el siguiente:

In [31]:
porcentaje_nulos = (cantidad_nulos / total_registros) * 100
porcentaje_nulos

ip,timestamp,request_method,url,http_version,status_code,size,user_agent
f64,f64,f64,f64,f64,f64,f64,f64
0.0,0.0,0.223421,0.0,0.0,0.223904,0.223904,0.223904


#### Limpieza de nulos
Vemos que hay una gran cantidad de celdas con valores en nulo, por lo que vamos a filtrar las filas que tienen valores vacíos en al menos una columna para poder visualizar algunos casos

In [32]:
from functools import reduce

mask = reduce(lambda a, b: a | b, [df[col].is_null() for col in df.columns])
rows_con_nulos = df.filter(mask)
rows_con_nulos.head(10)

ip,timestamp,request_method,url,http_version,status_code,size,user_agent
str,str,str,str,str,i64,i64,str
"""186.49.154.44""","""02/Jun/2013:16:24:20 -0300""",null,"""408""","""0""",null,null,null
"""66.249.82.10""","""02/Jun/2013:16:42:06 -0300""",null,"""408""","""0""",null,null,null
"""186.49.154.44""","""02/Jun/2013:16:51:04 -0300""",null,"""408""","""0""",null,null,null
"""186.49.13.241""","""02/Jun/2013:17:13:22 -0300""",null,"""408""","""0""",null,null,null
"""190.233.189.163""","""02/Jun/2013:17:59:33 -0300""",null,"""408""","""0""",null,null,null
"""190.233.189.163""","""02/Jun/2013:17:59:34 -0300""",null,"""408""","""0""",null,null,null
"""186.54.145.192""","""02/Jun/2013:18:27:14 -0300""",null,"""408""","""0""",null,null,null
"""186.54.145.192""","""02/Jun/2013:18:27:14 -0300""",null,"""408""","""0""",null,null,null
"""186.54.145.192""","""02/Jun/2013:18:27:15 -0300""",null,"""408""","""0""",null,null,null


Encontramos algo que nos llama la atención y es que están apareciendo algunos status code dentro de la columna de urls, lo podemos verificar con la siguiente consulta:

In [33]:
unique_status = df.select("url").unique()
print(unique_status.sort("url", descending=True))

shape: (28_677, 1)
┌─────────────────────────────────┐
│ url                             │
│ ---                             │
│ str                             │
╞═════════════════════════════════╡
│ http://www.sina.com.cn/         │
│ 408                             │
│ 200                             │
│ /wp-content/themes/Bold/timthu… │
│ /wp-conf.php                    │
│ …                               │
│ //modules/mod_btslideshow/imag… │
│ //admin/tting.php               │
│ /%7Bphocagallery%20view=catego… │
│ /                               │
│ *                               │
└─────────────────────────────────┘


Vamos a filtrar las filas que contienen códigos de estado en la columna `url` para que que valores tienen esas filas

In [34]:
status_codes = ["200","408"]
urls_with_status_codes = df.filter(pl.col("url").is_in(status_codes))

print(urls_with_status_codes)

shape: (464, 8)
┌──────────────┬──────────────┬──────────────┬─────┬─────────────┬─────────────┬──────┬────────────┐
│ ip           ┆ timestamp    ┆ request_meth ┆ url ┆ http_versio ┆ status_code ┆ size ┆ user_agent │
│ ---          ┆ ---          ┆ od           ┆ --- ┆ n           ┆ ---         ┆ ---  ┆ ---        │
│ str          ┆ str          ┆ ---          ┆ str ┆ ---         ┆ i64         ┆ i64  ┆ str        │
│              ┆              ┆ str          ┆     ┆ str         ┆             ┆      ┆            │
╞══════════════╪══════════════╪══════════════╪═════╪═════════════╪═════════════╪══════╪════════════╡
│ 186.49.154.4 ┆ 02/Jun/2013: ┆ null         ┆ 408 ┆ 0           ┆ null        ┆ null ┆ null       │
│ 4            ┆ 16:24:20     ┆              ┆     ┆             ┆             ┆      ┆            │
│              ┆ -0300        ┆              ┆     ┆             ┆             ┆      ┆            │
│ 66.249.82.10 ┆ 02/Jun/2013: ┆ null         ┆ 408 ┆ 0           ┆ null    

Investigando los casos null en profundidad nos encontramos con que son casos borde. Los de status 200 son casos con datos no legibles o anómalos y los de status 408 parecen ser una solicitud incompleta o malformada debido a un timeout en la solicitud, problemas de parte del cliente o error de red o conexión.

Este tipo de entradas en los logs a menudo se descartan en análisis de tráfico web ya que no proporcionan datos útiles sobre las solicitudes válidas. Sin embargo, pueden ser útiles para detectar problemas de red o errores en los clientes.

Por lo que vamos a filtrar las filas donde la columna `url` contenga los valores `200` o `408`

In [35]:
df_sin_nulos = df.filter(
    ~pl.col("url").str.contains('200') & ~pl.col("url").str.contains('408')
)

print(df_sin_nulos)
df_sin_nulos.write_csv("log_sin_nulos.csv")

shape: (203_725, 8)
┌─────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬──────┬────────────┐
│ ip          ┆ timestamp  ┆ request_me ┆ url        ┆ http_versi ┆ status_cod ┆ size ┆ user_agent │
│ ---         ┆ ---        ┆ thod       ┆ ---        ┆ on         ┆ e          ┆ ---  ┆ ---        │
│ str         ┆ str        ┆ ---        ┆ str        ┆ ---        ┆ ---        ┆ i64  ┆ str        │
│             ┆            ┆ str        ┆            ┆ str        ┆ i64        ┆      ┆            │
╞═════════════╪════════════╪════════════╪════════════╪════════════╪════════════╪══════╪════════════╡
│ 127.0.0.1   ┆ 02/Jun/201 ┆ OPTIONS    ┆ *          ┆ HTTP/1.0   ┆ 200        ┆ 167  ┆ Apache/2.2 │
│             ┆ 3:06:48:17 ┆            ┆            ┆            ┆            ┆      ┆ .14        │
│             ┆ -0300      ┆            ┆            ┆            ┆            ┆      ┆ (Ubuntu)   │
│             ┆            ┆            ┆            ┆            ┆    

Verificamos que ya no hayan más casos de datos nulos

In [36]:
cantidad_nulos = df_sin_nulos.null_count()
cantidad_nulos

ip,timestamp,request_method,url,http_version,status_code,size,user_agent
u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0


#### Limpieza de peticiones internas

Es común que el servidor web realice solicitudes internas a sí mismo para verificar su configuración, comprobar capacidades o realizar tareas de mantenimiento. El uso de * en las solicitudes OPTIONS puede estar asociado con estas operaciones internas.

La URL * en este contexto se refiere a una solicitud genérica que no está dirigida a una URL específica. En el caso de los métodos OPTIONS, * puede estar siendo usado para indicar que la solicitud no está dirigida a un recurso específico o es un marcador de posición para una solicitud interna del servidor.

Dado que las solicitudes provienen de 127.0.0.1 (localhost), parece que estas solicitudes son internas, probablemente generadas por el propio servidor para comprobar su configuración o para otros fines administrativos

Antes de eliminar esos registros, veamos cuántas peticiones internas hay en el dataset y qué porcentaje representan sobre la cantidad total de peticiones con los registros nulos ya filtrados.

In [37]:
peticiones_internas = df_sin_nulos.filter(
    (pl.col("ip") == "127.0.0.1") &
    (pl.col("request_method") == "OPTIONS") &
    (pl.col("url") == "*")
)

cantidad_peticiones_internas = peticiones_internas.shape[0]
total_registros = df_sin_nulos.shape[0]
porcentaje_peticiones_internas = (cantidad_peticiones_internas / total_registros) * 100

print(f"Cantidad de peticiones internas: {cantidad_peticiones_internas}")
print(f"Cantidad de registros (sin nulos): {total_registros}")
print(f"Porcentaje de peticiones internas: {porcentaje_peticiones_internas:.2f}%")

Cantidad de peticiones internas: 5386
Cantidad de registros (sin nulos): 203725
Porcentaje de peticiones internas: 2.64%


In [38]:
df_sin_nulos

ip,timestamp,request_method,url,http_version,status_code,size,user_agent
str,str,str,str,str,i64,i64,str
"""127.0.0.1""","""02/Jun/2013:06:48:17 -0300""","""OPTIONS""","""*""","""HTTP/1.0""",200,167,"""Apache/2.2.14 (Ubuntu) (intern…"
"""127.0.0.1""","""02/Jun/2013:06:48:17 -0300""","""OPTIONS""","""*""","""HTTP/1.0""",200,167,"""Apache/2.2.14 (Ubuntu) (intern…"
"""127.0.0.1""","""02/Jun/2013:06:48:17 -0300""","""OPTIONS""","""*""","""HTTP/1.0""",200,167,"""Apache/2.2.14 (Ubuntu) (intern…"
"""180.76.5.150""","""02/Jun/2013:06:49:39 -0300""","""GET""","""/index.php/component/jevents/s…","""HTTP/1.1""",200,8842,"""Mozilla/5.0 (compatible; Baidu…"
"""190.64.2.162""","""02/Jun/2013:06:51:40 -0300""","""GET""","""/robots.txt""","""HTTP/1.0""",200,718,"""agesic-crawler (Enterprise; T3…"
…,…,…,…,…,…,…,…
"""127.0.0.1""","""09/Jun/2013:06:43:46 -0300""","""OPTIONS""","""*""","""HTTP/1.0""",200,167,"""Apache/2.2.14 (Ubuntu) (intern…"
"""127.0.0.1""","""09/Jun/2013:06:43:46 -0300""","""OPTIONS""","""*""","""HTTP/1.0""",200,167,"""Apache/2.2.14 (Ubuntu) (intern…"
"""127.0.0.1""","""09/Jun/2013:06:43:46 -0300""","""OPTIONS""","""*""","""HTTP/1.0""",200,167,"""Apache/2.2.14 (Ubuntu) (intern…"


In [39]:
df_no_peticiones_internas = df_sin_nulos.filter(
    ~((pl.col("ip") == "127.0.0.1") & (pl.col("request_method") == "OPTIONS") & (pl.col("url") == "*"))
)

df_no_peticiones_internas.head(10)

ip,timestamp,request_method,url,http_version,status_code,size,user_agent
str,str,str,str,str,i64,i64,str
"""180.76.5.150""","""02/Jun/2013:06:49:39 -0300""","""GET""","""/index.php/component/jevents/s…","""HTTP/1.1""",200,8842,"""Mozilla/5.0 (compatible; Baidu…"
"""190.64.2.162""","""02/Jun/2013:06:51:40 -0300""","""GET""","""/robots.txt""","""HTTP/1.0""",200,718,"""agesic-crawler (Enterprise; T3…"
"""190.64.2.162""","""02/Jun/2013:06:51:40 -0300""","""GET""","""/index.php/component/jevents/m…","""HTTP/1.0""",200,9471,"""agesic-crawler (Enterprise; T3…"
"""190.64.2.162""","""02/Jun/2013:06:51:42 -0300""","""GET""","""/robots.txt""","""HTTP/1.0""",200,718,"""agesic-crawler (Enterprise; T3…"
"""190.64.2.162""","""02/Jun/2013:06:51:42 -0300""","""GET""","""/index.php/component/jevents/m…","""HTTP/1.0""",200,9518,"""agesic-crawler (Enterprise; T3…"
"""190.64.2.162""","""02/Jun/2013:06:51:44 -0300""","""GET""","""/robots.txt""","""HTTP/1.0""",200,718,"""agesic-crawler (Enterprise; T3…"
"""190.64.2.162""","""02/Jun/2013:06:51:44 -0300""","""GET""","""/index.php/component/jevents/m…","""HTTP/1.0""",200,9476,"""agesic-crawler (Enterprise; T3…"
"""190.64.2.162""","""02/Jun/2013:06:51:48 -0300""","""GET""","""/robots.txt""","""HTTP/1.0""",200,718,"""agesic-crawler (Enterprise; T3…"
"""190.64.2.162""","""02/Jun/2013:06:51:48 -0300""","""GET""","""/index.php/component/jevents/m…","""HTTP/1.0""",200,9490,"""agesic-crawler (Enterprise; T3…"


#### Limpieza de datos estáticos
Los archivos estáticos como .css o .png a menudo no contienen información crítica para el análisis de tráfico web o para la detección de problemas específicos en el comportamiento del usuario. Además muchas veces, los archivos estáticos son solicitados por bots o sistemas automatizados.

Datos innecesarios o erróneos pueden aumentar el tamaño de los datasets, haciendo que el procesamiento sea más lento. Limpiar los datos reduce el tamaño y mejora la eficiencia del procesamiento.

In [40]:
df_filtrado_datos_estaticos = filtrar_datos(df_no_peticiones_internas)

Registros 'jcemediabox': 5279 (2.55%)
Registros '.css': 18955 (9.15%)
Registros '.js': 22506 (10.86%)
Registros '.png': 22554 (10.88%)
Registros '.jpg': 7305 (3.53%)
Registros '.gif': 1859 (0.90%)
Registros 'favicon.ico': 892 (0.43%)


#### Limpieza de robots

In [41]:
df_filtrado_robots = filtrar_robots_y_crawlers(df_filtrado_datos_estaticos)

Registros 'Googlebot': 1840 (0.89%)
Registros 'Baiduspider': 1994 (0.96%)
Registros 'agesic-crawler': 28790 (13.89%)


In [42]:
cantidad_registros = df_filtrado_robots.select(pl.all().count())
cantidad_registros

ip,timestamp,request_method,url,http_version,status_code,size,user_agent
u32,u32,u32,u32,u32,u32,u32,u32
87819,87819,87819,87819,87819,87819,87819,87819


### Normalización

In [ ]:
df_limpio = formatear_fecha(df_filtrado)
df_limpio

ip,timestamp,request_method,url,http_version,status_code,size,user_agent
str,str,str,str,str,i64,i64,str
"""208.115.113.83…","""02-06-2013 10:…","""GET""","""/robots.txt""","""HTTP/1.1""",200,1151,"""Mozilla/5.0 (c…"
"""86.106.32.198""","""02-06-2013 10:…","""GET""","""/""","""HTTP/1.1""",200,48548,"""-"" """
"""157.55.36.37""","""02-06-2013 10:…","""GET""","""/robots.txt""","""HTTP/1.1""",200,1208,"""Mozilla/5.0 (c…"
"""157.55.36.37""","""02-06-2013 10:…","""GET""","""/""","""HTTP/1.1""",200,12030,"""Mozilla/5.0 (c…"
"""157.55.36.37""","""02-06-2013 10:…","""GET""","""/index.php/ser…","""HTTP/1.1""",200,8786,"""Mozilla/5.0 (c…"
"""157.55.36.37""","""02-06-2013 10:…","""GET""","""/index.php/inf…","""HTTP/1.1""",200,8523,"""Mozilla/5.0 (c…"
"""152.61.128.66""","""02-06-2013 10:…","""HEAD""","""/""","""HTTP/1.1""",403,163,"""Mozilla/5.0 (W…"
"""152.61.193.99""","""02-06-2013 10:…","""HEAD""","""/""","""HTTP/1.1""",403,163,"""Mozilla/5.0 (W…"
"""152.61.128.66""","""02-06-2013 10:…","""GET""","""/UYRV.cgi?VERS…","""HTTP/1.1""",200,3059,"""Mozilla/5.0 (W…"
